### Photonic lantern empirical identification procedure: pure linear algebra derivation

In [1]:
import numpy as np
N = 3
A, _ = np.linalg.qr(np.random.rand(N, N) + 1j * np.random.rand(N, N))
np.allclose(np.eye(N), np.dot(A, np.conj(A).T))

True

In the notebook with the optical setup, $A$ isn't unitary; I think there's some normalization that would get us to this, but none of the math depends on it being unitary so it's fine.

In [2]:
def query_A2(x):
    return np.abs(np.dot(A, x)) ** 2

test_query = np.zeros(N)
test_query[0] = 1.0
query_A2(test_query)

array([0.207004  , 0.68109071, 0.11190529])

###  Diagonal elements

We can easily identify the matrix up to phase differences:

In [3]:
A_recon = np.zeros((N,N), dtype=np.complex128)
diagonal_query = np.zeros(N)
for i in range(N):
    diagonal_query[:] = 0.0
    diagonal_query[i] = 1.0
    A_recon[:,i] = np.sqrt(query_A2(diagonal_query))
    
np.allclose(A_recon, np.abs(A))

True

### Phase differences

We don't have complete control over the remaining inputs, but we know we have $N(N-1)$ more inputs with random coefficients. Let's generate those:

In [12]:
queries = np.random.uniform(size=(N*(N-1), N), low=-1, high=1) + 1j * np.random.uniform(size=(N*(N-1), N), low=-1, high=1)
readings = np.array([query_A2(q) for q in queries])

We'll use this to make a linear system per port.

For each of the queries, we know its representation in the input basis (which I'll call the "fitting basis" for reasons that'll be clear with the full optical system), and we know its query under $A$ looks like

$ \text{query response} = |Aq_i|^2_p = |\sum_{k=1}^K A_{pk} q_{ki}|^2 $

Here, $i$ indexes the query number. A query is a linear combination of the fitting basis, which is indexed by $k$. $A$ is a map from the fitting basis to the electric fields on each port, indexed by $p$.

We'll need a general expansion of a sum of complex numbers under $||^2$ to do this; this is given by

$ |a + b|^2 = |a|^2 + |b|^2 + 2 |a| |b| \cos(\phi_a - \phi_b) $.

This lets us expand this expression as follows:

$ |Aq_i|^2_p = |\sum_{k=1}^K A_{pk} q_{ki}|^2 = \sum_{k=1}^K |A_{pk}|^2 |q_{ki}|^2 + 2\sum_{l=1}^K \sum_{m = 1}^{l-1} |A_{pl}| |A_{pm}| |q_{li}| |q_{mi}| \cos(\Phi_{pl} - \Phi_{pm} + \phi_{li} - \phi_{mi}) $

where for convenience I've set $\Phi_{ab} = \arg A_{ab}$ and $\phi_{ab} = \arg q_{ab}$. We're now iterating over the fitting basis twice and computing cross terms; these iterations are indexed by $l$ and $m$.

This gives us a linear system in the terms with the form $\cos(\Phi_{pl} - \Phi_{pm} + \phi_l - \phi_m)$. The issue here is $q_l, q_m$ don't remain constant, so we're really getting information about a different angle every time! This is easily resolved, though:

$ \cos(\Phi_{pl} - \Phi_{pm} + \phi_{li} - \phi_{mi}) = \cos(\Phi_{pl} - \Phi_{pm}) \cos(\phi_{li} - \phi_{mi}) - \sin(\Phi_{pl} - \Phi_{pm}) \sin(\phi_{li} - \phi_{mi})$

and we know all the $\phi_i$-s, so this extends our linear system even further! We have $N(N-1)$ variables (halved for permutation, doubled for both sine and cosine). There's some degree of redundancy because we know that $\cos^2 x = 1 - \sin^2 x$, but we'll leave that out. So we end up needing $N(N-1)$ queries.

First, let's subtract off the diagonal elements. 

In [14]:
# I'll do this vectorized later
cross_term_results = np.copy(readings)
for query_idx in range(N*(N-1)):
    for port_idx in range(N):
        diagonal = np.zeros(N, dtype=np.complex128)
        diagonal[port_idx] = queries[query_idx,port_idx]
        cross_term_results[query_idx,:] -= query_A2(diagonal)
        
cross_term_results /= 2

We have $N(N-1)$ examples overall, with $N$ ports each. What we've just generated is the $b$ matrices in $Cx = b$ for the $N$ problems we'll have to solve. So we need to make coefficient matrices as follows:

$ C_p \left[\begin{matrix} \cos(\Phi_{p1} - \Phi_{p2}) \\ \sin(\Phi_{p1} - \Phi_{p2}) \\ \cos(\Phi_{p1} - \Phi_{p3}) \\ \sin(\Phi_{p1} - \Phi_{p3}) \\ \cos(\Phi_{p2} - \Phi_{p3}) \\ \sin(\Phi_{p2} - \Phi_{p3}) \\ \vdots \end{matrix}\right] = b_p $ 

How do we populate $C_p$ to make this work? Let's say there's some mapping $(l, m) \iff n$ just so we don't clutter the index convention. Now $n$ is indexing cross terms in the fitting basis, but with just one number rather than two, so that we can build the linear system we need.

$ \begin{align} C_{p,2n,i} &= &|A_{pl}| |A_{pm}| |q_{li}| |q_{mi}| \cos(\phi_{li} - \phi_{mi}) \\
C_{p,2n+1,i} &= &-|A_{pl}| |A_{pm}| |q_{li}| |q_{mi}| \sin(\phi_{li} - \phi_{mi}) \end{align} $

In [21]:
# define the mapping between (l, m) and n
index_pairs = []
for lv in range(N):
    for mv in range(lv):
        index_pairs.append((lv, mv))

In [33]:
for port_idx in range(N):
    C = np.zeros((N*(N-1), N*(N-1)))
    for n in range(N*(N-1)//2):
        lv, mv = index_pairs[n]
        prefactors = np.abs(A_recon[port_idx,lv] * A_recon[port_idx,mv]) * np.abs(queries[:,lv]) * np.abs(queries[:,mv])
        angle_diff = np.angle(queries[:,lv]) - np.angle(queries[:,mv])
        C[2*n,:] = prefactors * np.cos(angle_diff)
        C[2*n+1,:] = -prefactors * np.sin(angle_diff)
        
    print(np.linalg.lstsq(C, cross_term_results[:,port_idx], rcond=None)[0])

[-0.71098039  2.95728877 -0.22469704 -0.93744061 -0.17989357  0.64086735]
[ 2.12276159 15.69099929  1.52446677 -2.63746445 -0.77932235 -2.76459708]
[ -0.59807003 -10.81384401   0.28553393   2.46865602   0.65620682
   0.47488436]


In [40]:
B_abs = np.random.uniform(size=(3,3))
B_phase = np.random.uniform(low=0, high=2*np.pi, size=(3,3))
B = B_abs * np.exp(1j * B_phase)

In [52]:
query = np.random.uniform(size=3) + 1j * np.random.uniform(size=3)
response = np.abs(np.dot(B, test_query)) ** 2

In [63]:
piecewise_built_response = np.zeros(3)
# add in the diagonal elements
for port_index in range(3):
    for fitting_basis_index in range(3):
        piecewise_built_response[port_index] += B_abs[port_index,fitting_basis_index] ** 2 * np.abs(query[fitting_basis_index]) ** 2

# add in the cross terms
for port_index in range(3):
    for lv in range(3):
        for mv in range(lv):
            piecewise_built_response[port_index] += 2 * B_abs[port_index,lv] * B_abs[port_index,mv] * np.abs(query[lv]) * np.abs(query[mv]) * np.cos(B_phase[port_index,lv] - B_phase[port_index,mv] + np.angle(query[lv]) - np.angle(query[mv]))

In [64]:
response

array([0.272161  , 0.66899227, 0.00258841])

In [65]:
piecewise_built_response

array([0.24548451, 0.08448673, 0.33705594])

In [211]:
def triangle_law(k):
    abs2 = 0.0
    for a in k:
        abs2 += abs(a) ** 2
        
    for i in range(len(k)):
        for j in range(i):
            a, b = k[i], k[j]
            abs2 += 2 * abs(a) * abs(b) * (np.real(a) * np.real(b) + np.imag(a) * np.imag(b)) / abs(a*b)
            
    return abs2

for _ in range(100):
    q = np.random.uniform(low=-1, high=1, size=10) + 1j * np.random.uniform(low=-1, high=1, size=10)
    assert np.isclose(triangle_law(q), abs(sum(q)) ** 2)

In [253]:
def triangle_law_diagonal_terms(k1, k2):
    abs2 = 0.0
    for (a, b) in zip(k1, k2):
        abs2 += abs(a*b) ** 2
        
    return abs2

def triangle_law_cross_terms(k1, k2):
    abs2 = 0.0
    for i in range(len(k1)):
        for j in range(i):
            a, b = k1[i] * k2[i], k1[j] * k2[j]
            phase_a, phase_b = np.angle(k1[i]) - np.angle(k1[j]), np.angle(k2[i]) - np.angle(k2[j])
            cos_factor = np.cos(phase_a + phase_b)
            abs2 += 2 * abs(a) * abs(b) * cos_factor
            
    return abs2

In [254]:
def triangle_law_and_multiply(k1, k2):
    return triangle_law_diagonal_terms(k1, k2) + triangle_law_cross_terms(k1, k2)

In [255]:
def slow_triangular_abs2_matmul(A, x):
    res = np.zeros(A.shape[0])
    for port_idx in range(A.shape[0]):
        res[port_idx] = triangle_law_and_multiply(A[port_idx,:], x)
            
    return res

def slow_triangular_cross_terms(A, x):
    res = np.zeros(A.shape[0])
    for port_idx in range(A.shape[0]):
        res[port_idx] = triangle_law_cross_terms(A[port_idx,:], x)
            
    return res

def slow_triangular_diagonal_terms(A, x):
    res = np.zeros(A.shape[0])
    for port_idx in range(A.shape[0]):
        res[port_idx] = triangle_law_diagonal_terms(A[port_idx,:], x)
            
    return res

In [256]:
def query_abs2(A, x):
    return np.abs(np.dot(A, x)) ** 2

def remove_diagonal_terms_from_query_abs2(A, x):
    N = len(x)
    res = query_abs2(A, x)
    for i in range(N):
        y = np.zeros(N, dtype=np.complex128)
        y[i] = x[i]
        res -= query_abs2(A, y)
        
    return res

In [257]:
def make_linear_system_to_find_matrix_phase(A):
    N = A.shape[0]
    for i in range(N*(N-1)):
        x = np.random.uniform(-1, 1, N) + 1j * np.random.uniform(-1, 1, N)
        res_cross = remove_diagonal_terms_from_query_abs2(B, x)
        assert np.allclose(
            res_cross,
            slow_triangular_cross_terms(B, x)
        )

In [258]:
make_linear_system_to_find_matrix_phase(B)